# Baseline network + pruning + quantization

This notebook trains the baseline network with exact same architecture as the one in paper [Recognition of handwritten Latin characters with diacritics using CNN](https://journals.pan.pl/dlibra/publication/136210/edition/119099/content/bulletin-of-the-polish-academy-of-sciences-technical-sciences-recognition-of-handwritten-latin-characters-with-diacritics-using-cnn-lukasik-edyta-charytanowicz-malgorzata-milosz-marek-tokovarov-michail-kaczorowska-monika-czerwinski-dariusz-zientarski-tomasz-2021-69-no-1?language=en)

Model architecture description:

"The architecture of the concrete CNN is shown in Fig. 2.
The input is a 32x32 binarized matrix. The input is then prop-
agated through 12 adaptable layers. First come two convolu-
tional layers having 32 filters with the size of 3x3 and stride 1.
Secondly, the output of the convolutional layer is fed to the
ReLU function. The output is down-sampled using a max-pool-
ing operation with a 2x2 stride. Next, the dropout technique is
used with the coefficient 0.25. The four operations (two con-
volutions, nonlinearity, max-pooling, and dropout) are repeated,
using 64 filters for the convolutional layers. The output of the
last layer is then flattened and fed through a fully connected
layer with 256 neurons and ReLU nonlinearities, dropped out
with the 0.25 coefficient, and a final output layer is fully con-
nected with a Softmax activation function. The Adam optimizer
and the cross-entropy loss function were used in the network. 
The output is a probability distribution over 89 classes."


Additionally, this network was pruned and quantized after training.



After training, model is serialized and uploaded to W&B project.

In [ ]:
! pip install tensorflow-model-optimization

In [1]:
import pathlib
import shutil
import os
import time
import datetime
import numpy as np
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import matplotlib.pyplot as plt
import zipfile

from typing import List

def load_data(run, artifact_name = "phcd_paper_splits_tfds") -> List[tf.data.Dataset]:
    """
    Downloads datasets from a wandb artifact and loads them into a list of tf.data.Datasets.
    """

    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(
        f"./artifacts/{artifact.name.replace(':', '-')}"
    ).resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()

    # if tf.__version__ minor is less than 10, use
    # tf.data.experimental.load instead of tf.data.Dataset.load

    if int(tf.__version__.split(".")[1]) < 10:
        load_function = tf.data.experimental.load
    else:
        load_function = tf.data.Dataset.load
    
    output_list = []
    for split in ["train", "test", "val"]:
        ds = load_function(str(artifact_dir / split), compression="GZIP")
        output_list.append(ds)
    
    return output_list

def get_readable_class_labels(subset = 'phcd_paper'):
    if subset == 'phcd_paper':
        return ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c',
        'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
        'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C',
        'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P',
        'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'ą', 'ć', 'ę',
        'ł', 'ń', 'ó', 'ś', 'ź', 'ż', 'Ą', 'Ć', 'Ę', 'Ł', 'Ń', 'Ó', 'Ś',
        'Ź', 'Ż', '+', '-', ':', ';', '$', '!', '?', '@', '.']
    elif subset == 'uppercase':
        return ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
        'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'Ą', 'Ć', 
        'Ę', 'Ł', 'Ń', 'Ó', 'Ś', 'Ź', 'Ż']
    elif subset == 'lowercase':
        return ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
        'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ą', 'ć',
        'ę', 'ł', 'ń', 'ó', 'ś', 'ź', 'ż']
    elif subset == 'numbers':
        return ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    elif subset == 'uppercase_no_diacritics':
        return ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
        'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    elif subset == 'lowercase_no_diacritics':
        return ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
        'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

def calculate_accuracy_per_class(model, test_dataset, test_dataset_name):
    '''
    Calculates the accuracy per class for a given model and test dataset.

    Returns dict with class labels as keys and accuracy as values.
    '''
        
    y_pred = model.predict(test_dataset)
    y_pred = np.argmax(y_pred, axis=1)
    # get labels
    y_true = test_dataset.map(lambda x, y: y).as_numpy_iterator()
    y_true = np.concatenate(list(y_true))
    # calculate accuracy per class
    labels = get_readable_class_labels(test_dataset_name)
    class_accuracy = np.zeros(len(labels))
    for i, label in enumerate(labels):
        class_accuracy[i] = np.sum(y_pred[y_true == i] == i) / np.sum(y_true == i)
    return { label: acc for label, acc in zip(labels, class_accuracy) }
    

def plot_accuracy_per_class(class_accuracy_dict):
    plt.figure(figsize=(10, 5))
    labels = list(class_accuracy_dict.keys())
    class_accuracy = list(class_accuracy_dict.values())
    plt.bar(labels, class_accuracy)
    plt.xticks(labels)
    plt.xlabel("Class")
    plt.ylabel("Accuracy")
    plt.title("Accuracy per class")
    plt.show()


def accuracy_table(class_accuracy_dict):
    labels = list(class_accuracy_dict.keys())
    class_accuracy = list(class_accuracy_dict.values())
    return wandb.Table(columns=["Class", "Accuracy"], data=list(zip(labels, class_accuracy)))

def get_number_of_classes(ds: tf.data.Dataset) -> int:
    """
    Returns the number of classes in a dataset.
    """
    labels_iterator= ds.map(lambda x, y: y).as_numpy_iterator()
    labels = np.concatenate(list(labels_iterator))
    return len(np.unique(labels))

def get_number_of_examples(ds: tf.data.Dataset) -> int:
    """
    Returns the number of examples in a dataset.
    """
    return sum(1 for _ in ds)

def preprocess_dataset(ds: tf.data.Dataset, batch_size: int, cache: bool = True) -> tf.data.Dataset:
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))  # normalize
    ds = ds.unbatch().batch(batch_size)
    if cache:
        ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def calculate_model_compressed_size_on_disk(path: str) -> int:
    compressed_path = path + ".zip"
    with zipfile.ZipFile(compressed_path, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(path)
    return pathlib.Path(compressed_path).stat().st_size    

def calculate_model_num_parameters(model: tf.keras.Model) -> int:
    return model.count_params()

def calculate_model_flops(summary) -> float:
    # from run.summary get GFLOPs or GFLOPS whichever is available
    if "GFLOPs" in summary.keys():
        return summary.get("GFLOPs")
    elif "GFLOPS" in summary.keys():
        return summary.get("GFLOPS")
    else:
        return 0

def calculate_model_throughput(model, test_dataset, batch_size) -> float:
    '''
    Calculates the average throughput of a model over 50 batch predictions of 100 batches, in images per second.
    '''
    output = []
    for _ in range(50):
        start = time.time()
        model.predict(test_dataset.take(100))
        end = time.time()
        output.append((100 * batch_size) / (end - start))
    return np.mean(output)

def plot_history(history, title):
    plt.figure(figsize=(15,7))
    plt.suptitle(title)
    
    plt.subplot(121)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='val')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend()
    
    plt.subplot(122)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend()

2023-02-03 12:36:03.332400: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Available devices: ", tf.config.list_physical_devices())

Num GPUs Available:  1
Available devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-02-03 12:36:08.482861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-03 12:36:08.500311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-03 12:36:08.500689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.


# Baseline model

In [3]:
defaults = dict(
    batch_size=32*2,
    epochs=60,    
    optimizer="adam"
)

artifact_base_name = "phcd_paper"
model_name = f"baseline-{artifact_base_name}"
artifact_name = f"{artifact_base_name}_splits_tfds" # "phcd_paper_splits_tfds
run = wandb.init(project="master-thesis", job_type="training", name=model_name, config=defaults, tags=[artifact_name])
    
# hyperparameters
epochs = wandb.config.epochs
bs = wandb.config.batch_size

ds_train, ds_test, ds_val = load_data(run, artifact_name=artifact_name)

num_classes = get_number_of_classes(ds_val)

ds_train = preprocess_dataset(ds_train, batch_size=bs)
ds_val = preprocess_dataset(ds_val, batch_size=bs)
ds_test = preprocess_dataset(ds_test, batch_size=bs, cache=False)

'''
The architecture of the concrete CNN is shown in Fig. 2.
The input is a 32x32 binarized matrix. 
The input is then propagated through 12 adaptable layers. 
First come two convolutional layers having 32 filters with the size of 3x3 and stride 1.
Secondly, the output of the convolutional layer is fed to the
ReLU function. The output is down-sampled using a max-pool-
ing operation with a 2x2 stride. Next, the dropout technique is
used with the coefficient 0.25. The four operations (two con-
volutions, nonlinearity, max-pooling, and dropout) are repeated,
using 64 filters for the convolutional layers. The output of the
last layer is then flattened and fed through a fully connected
layer with 256 neurons and ReLU nonlinearities, dropped out
with the 0.25 coefficient, and a final output layer is fully con-
nected with a Softmax activation function. The Adam optimizer
and the cross-entropy loss function were used in the network
[24]. The output is a probability distribution over 89 classes.
'''

model = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(32, 32, 1)),
        
        # 2 Convolutional layers with 32 filters, 3x3 size, and stride 1
        tf.keras.layers.Conv2D(32, (3, 3), strides=1, activation='relu'),
        tf.keras.layers.Conv2D(32, (3, 3), strides=1, activation='relu'),
        
        # Max-pooling operation with 2x2 stride
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        # Dropout with coefficient 0.25
        tf.keras.layers.Dropout(0.25),
        
        # Repeat above 4 operations using 64 filters
        tf.keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
        tf.keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        tf.keras.layers.Dropout(0.25),
        
        # Flatten the output and feed through fully connected layer with 256 neurons
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),

        # Dropout with coefficient 0.25
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.summary()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gratkadlafana. Use `wandb login --relogin` to force relogin


wandb:   9 of 9 files downloaded.  
2023-02-03 12:36:15.600317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 12:36:15.601464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-03 12:36:15.601747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-03 12:36:15.601987: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/b

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)      

In [ ]:
history = model.fit(
    ds_train,
    epochs=wandb.config.epochs,
    validation_data=ds_val,
    callbacks=[
        WandbCallback(
            compute_flops=True, 
            save_model=False, 
            log_weights=False, 
            log_gradients=False
        )
    ],
)

plot_history(history, "Baseline")
tf.keras.models.save_model(model, 'model_baseline.h5', include_optimizer=False)

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Epoch 1/60


2023-02-03 12:36:18.107612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-03 12:36:18.107652: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-02-03 12:36:18.107859: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-02-03 12:36:18.108817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-03 12:36:18.109088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-03 12:36:18.109318: I tensor

1808/1808 [==============================] - 33s 15ms/step - loss: 0.3986 - accuracy: 0.8831 - val_loss: 0.1428 - val_accuracy: 0.9572
Epoch 2/60
1808/1808 [==============================] - 24s 13ms/step - loss: 0.1776 - accuracy: 0.9473 - val_loss: 0.1159 - val_accuracy: 0.9644
Epoch 3/60
1142/1808 [=================>............] - ETA: 7s - loss: 0.1564 - accuracy: 0.9530

In [ ]:
# evaluate model on ds_test and log to wandb
test_loss_before, test_acc_before = model.evaluate(ds_test)

# calculate model size on disk, flops and number of parameters
flops = calculate_model_flops(wandb.run.summary)
num_parameters = calculate_model_num_parameters(model)
compressed_disk_size = calculate_model_compressed_size_on_disk('model_baseline.h5')
throughput = calculate_model_throughput(model, ds_test, wandb.config.batch_size)
class_acc = calculate_accuracy_per_class(model, ds_test, artifact_base_name)

data_to_log = {
    "test_loss": test_loss_before, 
    "test_accuracy": test_acc_before, 
    "num_parameters": num_parameters,
    "compressed_disk_size": compressed_disk_size, 
    "model_flops": flops,
    "throughput": throughput,
    "accuracy_per_class": accuracy_table(class_acc)
    }
plot_accuracy_per_class(class_acc)
print(data_to_log)
wandb.log(data_to_log)

# upload model to wandb
wandb.save('model_baseline.h5')
run.finish()

In [ ]:
raise Exception("Stop here")

# Quantization - allows for better accuracy

In [ ]:
run = wandb.init(project="master-thesis", job_type="training", name=f"{model_name}_quantized", config=defaults, tags=[artifact_name, "optimized"])

quantize_model = tfmot.quantization.keras.quantize_model

quant_epochs = 2
base_model = tf.keras.models.load_model('model_baseline.h5')

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(base_model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

In [ ]:
history = q_aware_model.fit(
    ds_train,
    epochs=quant_epochs,
    validation_data=ds_val,
    callbacks=[
        WandbCallback(
            compute_flops=True, 
            save_model=False, 
            log_weights=False, 
            log_gradients=False
        )
    ],
)


plot_history(history, "Quantized")
tf.keras.models.save_model(q_aware_model, 'model_quantized.h5', include_optimizer=False)

In [ ]:
# evaluate model on ds_test and log to wandb
test_loss_quantize, test_acc_quantize = model.evaluate(ds_test)

# calculate model size on disk, flops and number of parameters
flops = calculate_model_flops(wandb.run.summary)
num_parameters = calculate_model_num_parameters(q_aware_model)
compressed_disk_size = calculate_model_compressed_size_on_disk('model_quantized.h5')
throughput = calculate_model_throughput(q_aware_model, ds_test, wandb.config.batch_size)
class_acc = calculate_accuracy_per_class(q_aware_model, ds_test, artifact_base_name)

data_to_log = {
    "test_loss": test_loss_quantize, 
    "test_accuracy": test_acc_quantize, 
    "num_parameters": num_parameters,
    "compressed_disk_size": compressed_disk_size, 
    "model_flops": flops,
    "throughput": throughput,
    "accuracy_per_class": accuracy_table(class_acc)
    }
plot_accuracy_per_class(class_acc)
print(data_to_log)
wandb.log(data_to_log)
run.finish()

# Pruning model - allows for smaller size (after compression)

In [ ]:
run = wandb.init(project="master-thesis", job_type="training", name=f"{model_name}_pruned", config=defaults, tags=[artifact_name, "optimized"])

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

prune_epochs = 4
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=0, frequency=100)
  }

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

base_model = tf.keras.models.load_model('model_baseline.h5')
model_sparse = prune_low_magnitude(base_model, **pruning_params)

# Use smaller learning rate for fine-tuning
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

model_sparse.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

model_sparse.summary()

In [ ]:
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    WandbCallback(
            compute_flops=True, 
            save_model=False, 
            log_weights=False, 
            log_gradients=False
        )
]

history = model_sparse.fit(
    ds_train,
    epochs=prune_epochs,
    validation_data=ds_val,
    callbacks=callbacks
)

plot_history(history, "Pruned")
tf.keras.models.save_model(model_sparse, 'model_sparse.h5', include_optimizer=False)

In [ ]:
# evaluate model on ds_test and log to wandb
test_loss_prune, test_acc_prune = model_sparse.evaluate(ds_test)

# calculate model size on disk, flops and number of parameters
flops = calculate_model_flops(wandb.run.summary)
num_parameters = calculate_model_num_parameters(model_sparse)
compressed_disk_size = calculate_model_compressed_size_on_disk('model_sparse.h5')
throughput = calculate_model_throughput(model_sparse, ds_test, wandb.config.batch_size)
class_acc = calculate_accuracy_per_class(model_sparse, ds_test, artifact_base_name)

data_to_log = {
    "test_loss": test_loss_prune, 
    "test_accuracy": test_acc_prune, 
    "num_parameters": num_parameters,
    "compressed_disk_size": compressed_disk_size, 
    "model_flops": flops,
    "throughput": throughput,
    "accuracy_per_class": accuracy_table(class_acc)
    }
plot_accuracy_per_class(class_acc)
print(data_to_log)
wandb.log(data_to_log)
run.finish()

# PQAT - pruning preserving quantization aware training

In [ ]:
run = wandb.init(project="master-thesis", job_type="training", name=f"{model_name}_pqat", config=defaults, tags=[artifact_name, "optimized"])

# PQAT
stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(model_sparse)
quant_aware_annotate_model = tfmot.quantization.keras.quantize_annotate_model(stripped_pruned_model)
pqat_model = tfmot.quantization.keras.quantize_apply(
              quant_aware_annotate_model,
              tfmot.experimental.combine.Default8BitPrunePreserveQuantizeScheme())

pqat_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir='./prune_summary'),
    WandbCallback(
            compute_flops=True, 
            save_model=False, 
            log_weights=False, 
            log_gradients=False
        )
]

history = pqat_model.fit(
    ds_train,
    epochs=prune_epochs,
    validation_data=ds_val,
    callbacks=callbacks,
)
plot_history(history, "Pruned model + PQAT")
tf.keras.models.save_model(pqat_model, 'model_quantized_sparse.h5', include_optimizer=False)

In [ ]:
# evaluate model on ds_test and log to wandb
test_loss_pqat, test_acc_pqat = pqat_model.evaluate(ds_test)

# calculate model size on disk, flops and number of parameters
flops = calculate_model_flops(wandb.run.summary)
num_parameters = calculate_model_num_parameters(pqat_model)
compressed_disk_size = calculate_model_compressed_size_on_disk('model_quantized_sparse.h5')
throughput = calculate_model_throughput(pqat_model, ds_test, wandb.config.batch_size)
class_acc = calculate_accuracy_per_class(pqat_model, ds_test, artifact_base_name)

data_to_log = {
    "test_loss": test_loss_pqat, 
    "test_accuracy": test_acc_pqat, 
    "num_parameters": num_parameters,
    "compressed_disk_size": compressed_disk_size, 
    "model_flops": flops,
    "throughput": throughput,
    "accuracy_per_class": accuracy_table(class_acc)
    }
plot_accuracy_per_class(class_acc)
print(data_to_log)
wandb.log(data_to_log)
run.finish()

# Summary & Tensorboard

In [ ]:
print('Baseline test accuracy:', test_acc_before) 
print('Quantized test accuracy:', test_acc_quantize)
print('Pruned test accuracy:', test_acc_prune)
print('PQAT test accuracy:', test_acc_pqat)

In [ ]:
print('Zipped model files size:\n')
b_size = os.path.getsize('model_baseline.h5.zip')
q_size = os.path.getsize('model_quantized.h5.zip')
s_size = os.path.getsize('model_sparse.h5.zip')
q_s_size = os.path.getsize('model_quantized_sparse.h5.zip')

print('Baseline: {} bytes'.format(b_size))
print('Quantized: {} bytes'.format(q_size))
print('Sparse:   {} bytes'.format(s_size))
print('PQAT:     {} bytes'.format(q_s_size))

print(f'Quantized model is {((b_size-q_size)/b_size * 100):.2f}% smaller')
print(f'Pruned model is {((b_size-s_size)/b_size * 100):.2f}% smaller')
print(f'PQAT model is {((b_size-q_s_size)/b_size * 100):.2f}% smaller')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir prune_summary